In [1]:
# Import the required modules
import pandas as pd
pd.set_option('display.max_columns', None)
import numpy as np

# visualization
import matplotlib.pyplot as plt
import seaborn as sns
import hvplot.pandas

# Machine Learning
from sklearn.cluster import KMeans, AgglomerativeClustering, Birch
from sklearn.metrics import silhouette_score, calinski_harabasz_score
from sklearn.preprocessing import StandardScaler, MinMaxScaler, OneHotEncoder, OrdinalEncoder
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA

# suppress warnings
import warnings
warnings.filterwarnings('ignore')

In [2]:
# Load the data into a Pandas DataFrame
df_market_data = pd.read_csv(
    "Resources/crypto_market_data.csv",
    index_col="coin_id")

# Display sample data
df_market_data.head(10)

,price_change_percentage_24h,price_change_percentage_7d,price_change_percentage_14d,price_change_percentage_30d,price_change_percentage_60d,price_change_percentage_200d,price_change_percentage_1y
coin_id,,,,,,,
bitcoin,1.08388,7.60278,6.57509,7.67258,-3.25185,83.51840,37.51761
ethereum,0.22392,10.38134,4.80849,0.13169,-12.88890,186.77418,101.96023
tether,-0.21173,0.04935,0.00640,-0.04237,0.28037,-0.00542,0.01954
ripple,-0.37819,-0.60926,2.24984,0.23455,-17.55245,39.53888,-16.60193
bitcoin-cash,2.90585,17.09717,14.75334,15.74903,-13.71793,21.66042,14.49384
binancecoin,2.10423,12.85511,6.80688,0.05865,36.33486,155.61937,69.69195
chainlink,-0.23935,20.69459,9.30098,-11.21747,-43.69522,403.22917,325.13186
cardano,0.00322,13.99302,5.55476,10.10553,-22.84776,264.51418,156.09756
litecoin,-0.06341,6.60221,7.28931,1.21662,-17.23960,27.49919,-12.66408


In [3]:
# Generate summary statistics
df_market_data.describe()

,price_change_percentage_24h,price_change_percentage_7d,price_change_percentage_14d,price_change_percentage_30d,price_change_percentage_60d,price_change_percentage_200d,price_change_percentage_1y
count,41.000000,41.000000,41.000000,41.000000,41.000000,41.000000,41.000000
mean,-0.269686,4.497147,0.185787,1.545693,-0.094119,236.537432,347.667956
std,2.694793,6.375218,8.376939,26.344218,47.365803,435.225304,1247.842884
min,-13.527860,-6.094560,-18.158900,-34.705480,-44.822480,-0.392100,-17.567530
25%,-0.608970,0.047260,-5.026620,-10.438470,-25.907990,21.660420,0.406170
50%,-0.063410,3.296410,0.109740,-0.042370,-7.544550,83.905200,69.691950
75%,0.612090,7.602780,5.510740,4.578130,0.657260,216.177610,168.372510
max,4.840330,20.694590,24.239190,140.795700,223.064370,2227.927820,7852.089700


In [4]:
# Plot your data to see what's in your DataFrame
df_market_data.hvplot.line(
    width=800,
    height=400,
    rot=90
)

:NdOverlay   [Variable]
   :Curve   [coin_id]   (value)

---

### Prepare the Data

In [5]:
# Use the `StandardScaler()` module from scikit-learn to normalize the data from the CSV file
# Select features to scale
features = ['price_change_percentage_24h', 'price_change_percentage_7d']
df_to_scale = df_market_data[features]

# Initialize StandardScaler
scaler = StandardScaler()

# Fit the scaler and transform the data
scaled_data = scaler.fit_transform(df_to_scale)

# Create a new DataFrame with the scaled data
df_scaled = pd.DataFrame(scaled_data, columns=features, index=df_market_data.index)

# Display the first few rows
df_scaled.head()

,price_change_percentage_24h,price_change_percentage_7d
coin_id,,
bitcoin,0.508529,0.493193
ethereum,0.185446,0.934445
tether,0.021774,-0.706337
ripple,-0.040764,-0.810928
bitcoin-cash,1.193036,2.000959


In [6]:
# Create a DataFrame with the scaled data
df_scaled = pd.DataFrame(scaled_data, columns=features)

# Copy the crypto names (coin_id is the index from original DataFrame)
df_scaled['coin_id'] = df_market_data.index

# Set the coin_id column as index
df_scaled = df_scaled.set_index('coin_id')

# Display the scaled DataFrame
df_scaled.head()

,price_change_percentage_24h,price_change_percentage_7d
coin_id,,
bitcoin,0.508529,0.493193
ethereum,0.185446,0.934445
tether,0.021774,-0.706337
ripple,-0.040764,-0.810928
bitcoin-cash,1.193036,2.000959


---

### Find the Best Value for k Using the Original Scaled DataFrame.

In [7]:
# Create a list with the number of k-values from 1 to 11
k_values = list(range(1, 12))
k_values

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]

In [8]:
# Create an empty list to store the inertia values
inertia = []

# Create a for loop to compute the inertia with each possible value of k
for k in k_values:
    # 1. Create a KMeans model using k clusters
    model = KMeans(n_clusters=k, random_state=1)

    # 2. Fit the model to the scaled data
    model.fit(df_scaled)

    # 3. Append the inertia to the list
    inertia.append(model.inertia_)

In [9]:
# Create a dictionary with the data to plot the Elbow curve
elbow_data = {
    "k": k_values,
    "inertia": inertia
}

# Create a DataFrame with the data
df_elbow = pd.DataFrame(elbow_data)

# Display the DataFrame (optional)
df_elbow

,k,inertia
0,1,82.000000
1,2,50.727034
2,3,26.792512
3,4,19.840983
4,5,14.019787
5,6,10.081393
6,7,8.872959
7,8,7.035661
8,9,5.534640
9,10,4.475534


In [10]:
# Plot a line chart with all the inertia values computed with
# the different values of k to visually identify the optimal value for k.
# Plot the Elbow Curve
df_elbow.hvplot.line(
    x='k',
    y='inertia',
    title='Elbow Curve',
    xlabel='Number of Clusters (k)',
    ylabel='Inertia',
    xticks=k_values,
    width=600,
    height=400
)

:Curve   [k]   (inertia)

#### Answer the following question: 

**Question:** What is the best value for `k`?

**Answer:**  Based on the Elbow Curve, the optimal value for k is 3. This is the point where the inertia significantly decreases up to k=3, and then begins to level off, indicating diminishing returns for additional clusters.

---

### Cluster Cryptocurrencies with K-means Using the Original Scaled DataFrame

In [11]:
# Initialize the K-Means model using the best value for k
k = 3  # Best value from the elbow method
kmeans_model = KMeans(n_clusters=k, random_state=1)

In [12]:
# Fit the K-Means model using the scaled DataFrame
kmeans_model.fit(df_scaled)

KMeans(n_clusters=3, random_state=1)

In [13]:
# Predict the clusters to group the cryptocurrencies using the scaled DataFrame
predicted_clusters = kmeans_model.predict(df_scaled)

# Print the resulting array of cluster values
predicted_clusters

array([0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0,
       1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 2, 0, 1, 0, 1, 1, 0, 1, 1])

In [14]:
# Create a copy of the scaled DataFrame
df_clusters = df_scaled.copy()

In [15]:
# Add a new column to the copy of the scaled DataFrame with the predicted clusters
df_clusters['cluster'] = predicted_clusters

# Display the copy of the scaled DataFrame
df_clusters.head()

,price_change_percentage_24h,price_change_percentage_7d,cluster
coin_id,,,
bitcoin,0.508529,0.493193,0
ethereum,0.185446,0.934445,0
tether,0.021774,-0.706337,1
ripple,-0.040764,-0.810928,1
bitcoin-cash,1.193036,2.000959,0


In [16]:
# Create a scatter plot using hvPlot by setting
# `x="price_change_percentage_24h"` and `y="price_change_percentage_7d"`.
# Color the graph points with the labels found using K-Means and
# add the crypto name in the `hover_cols` parameter to identify
# the cryptocurrency represented by each data point.
df_clusters.hvplot.scatter(
    x='price_change_percentage_24h',
    y='price_change_percentage_7d',
    by='cluster',
    hover_cols=['coin_id'],
    title='Crypto Clusters by 24h and 7d Price Change',
    width=700,
    height=500
)

:NdOverlay   [cluster]
   :Scatter   [price_change_percentage_24h]   (price_change_percentage_7d,coin_id)

---

### Optimize Clusters with Principal Component Analysis.

In [24]:
# Create a PCA model instance and set `n_components=3`.
pca = PCA(n_components=2)

In [25]:
# Use the PCA model with fit_transform to reduce the original scaled DataFrame
pca_data = pca.fit_transform(df_scaled)

# View the scaled PCA data (as a NumPy array)
pca_data[:5]

array([[-0.70832473, -0.0108444 ],
       [-0.79188247,  0.52962238],
       [ 0.48405906, -0.51485209],
       [ 0.60223751, -0.54458796],
       [-2.25849563,  0.57128782]])

In [26]:
# Retrieve the explained variance to determine how much information
# can be attributed to each principal component.
explained_variance = pca.explained_variance_ratio_

# Print explained variance for each component
print("Explained Variance per Principal Component:")
print(f"PC1: {explained_variance[0]:.4f}")
print(f"PC2: {explained_variance[1]:.4f}")

# Print the total explained variance
print(f"Total Explained Variance: {explained_variance.sum():.4f}")

Explained Variance per Principal Component:
PC1: 0.5848
PC2: 0.4152
Total Explained Variance: 1.0000


#### Answer the following question: 

**Question:** What is the total explained variance of the three principal components?

**Answer:** We only had two original features, so PCA could only generate two components. The total explained variance of these two components is 1.0000, or 100%. This means no information was lost during dimensionality reduction.

In [27]:
# Create a new DataFrame with the PCA data
df_pca = pd.DataFrame(
    pca_data,
    columns=['PC1', 'PC2']
)

# Copy the crypto names from the original scaled DataFrame
df_pca['coin_id'] = df_scaled.index

# Set the coin_id column as index
df_pca = df_pca.set_index('coin_id')

# Display the scaled PCA DataFrame
df_pca.head()

,PC1,PC2
coin_id,,
bitcoin,-0.708325,-0.010844
ethereum,-0.791882,0.529622
tether,0.484059,-0.514852
ripple,0.602238,-0.544588
bitcoin-cash,-2.258496,0.571288


---

### Find the Best Value for k Using the Scaled PCA DataFrame

In [28]:
# Create a list with the number of k-values from 1 to 11
k_values_pca = list(range(1, 12))

# Optional: Display the list
print(k_values_pca)

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]


In [29]:
# Create an empty list to store the inertia values
inertia_pca = []

# Create a for loop to compute the inertia with each possible value of k
# Inside the loop:
# Loop through each value of k and compute inertia
for k in k_values_pca:
    # 1. Create a KMeans model with k clusters
    model = KMeans(n_clusters=k, random_state=1)

    # 2. Fit the model to the PCA data
    model.fit(df_pca)

    # 3. Append the inertia to the list
    inertia_pca.append(model.inertia_)

In [30]:
# Create a dictionary with the data to plot the Elbow curve
elbow_data_pca = {
    "k": k_values_pca,
    "inertia": inertia_pca
}

# Create a DataFrame with the data
df_elbow_pca = pd.DataFrame(elbow_data_pca)

# Optional: View the DataFrame
df_elbow_pca

,k,inertia
0,1,82.000000
1,2,50.727034
2,3,26.792512
3,4,19.840983
4,5,14.019787
5,6,10.081393
6,7,8.872959
7,8,7.035661
8,9,5.534640
9,10,4.475534


In [31]:
# Plot a line chart with all the inertia values computed with
# the different values of k to visually identify the optimal value for k.
# Plot the Elbow Curve for the PCA data
df_elbow_pca.hvplot.line(
    x='k',
    y='inertia',
    title='Elbow Curve (PCA Data)',
    xlabel='Number of Clusters (k)',
    ylabel='Inertia',
    xticks=k_values_pca,
    width=600,
    height=400
)

:Curve   [k]   (inertia)

#### Answer the following questions: 

* **Question:** What is the best value for `k` when using the PCA data?

  * **Answer:**  The best value for k when using the PCA-transformed data is 3. This was determined by identifying the "elbow point" on the inertia curve, where the rate of decrease in inertia levels off.


* **Question:** Does it differ from the best k value found using the original data?

  * **Answer:** No, it does not differ. The best k value for both the original scaled data and the PCA-transformed data is 3, indicating that dimensionality reduction preserved the cluster structure in the data.

### Cluster Cryptocurrencies with K-means Using the Scaled PCA DataFrame

In [32]:
# Initialize the K-Means model using the best value for k
k_pca = 3  # Based on the elbow method
kmeans_pca = KMeans(n_clusters=k_pca, random_state=1)

In [33]:
# Fit the K-Means model using the PCA data
kmeans_pca.fit(df_pca)

KMeans(n_clusters=3, random_state=1)

In [34]:
# Predict the clusters to group the cryptocurrencies using the scaled PCA DataFrame
predicted_pca_clusters = kmeans_pca.predict(df_pca)

# Print the resulting array of cluster values
print(predicted_pca_clusters)

[0 0 1 1 0 0 0 0 0 1 1 1 1 0 1 0 1 1 0 1 1 0 1 1 1 1 0 1 0 1 1 1 2 0 1 0 1
 1 0 1 1]


In [35]:
# Create a copy of the scaled PCA DataFrame
df_pca_clusters = df_pca.copy()

# Add a new column with the predicted clusters
df_pca_clusters['cluster'] = predicted_pca_clusters

# Display the DataFrame
df_pca_clusters.head()

,PC1,PC2,cluster
coin_id,,,
bitcoin,-0.708325,-0.010844,0
ethereum,-0.791882,0.529622,0
tether,0.484059,-0.514852,1
ripple,0.602238,-0.544588,1
bitcoin-cash,-2.258496,0.571288,0


In [36]:
# Create a scatter plot using hvPlot by setting
# `x="PC1"` and `y="PC2"`.
# Color the graph points with the labels found using K-Means and
# add the crypto name in the `hover_cols` parameter to identify
# the cryptocurrency represented by each data point.
df_pca_clusters.hvplot.scatter(
    x='PC1',
    y='PC2',
    by='cluster',
    hover_cols=['coin_id'],
    title='Crypto Clusters (PCA Data)',
    width=700,
    height=500
)

:NdOverlay   [cluster]
   :Scatter   [PC1]   (PC2,coin_id)

### Visualize and Compare the Results

In this section, you will visually analyze the cluster analysis results by contrasting the outcome with and without using the optimization techniques.

In [37]:
# Composite plot to contrast the Elbow curves
elbow_plot = df_elbow.hvplot.line(
    x='k',
    y='inertia',
    title='Elbow Curve (Original vs PCA)',
    xlabel='Number of Clusters (k)',
    ylabel='Inertia',
    xticks=list(range(1, 12)),
    line_width=2,
    color='blue',
    label='Original Data'
)

elbow_pca_plot = df_elbow_pca.hvplot.line(
    x='k',
    y='inertia',
    line_width=2,
    color='orange',
    label='PCA Data'
)

# Show both curves side-by-side on the same plot
elbow_plot + elbow_pca_plot

:Layout
   .Curve.Original_Data :Curve   [k]   (inertia)
   .Curve.PCA_Data      :Curve   [k]   (inertia)

In [38]:
# Composite plot to contrast the clusters
original_cluster_plot = df_clusters.hvplot.scatter(
    x='price_change_percentage_24h',
    y='price_change_percentage_7d',
    by='cluster',
    hover_cols=['coin_id'],
    title='Clusters (Original Scaled Data)',
    width=500,
    height=500
)

# PCA feature cluster plot
pca_cluster_plot = df_pca_clusters.hvplot.scatter(
    x='PC1',
    y='PC2',
    by='cluster',
    hover_cols=['coin_id'],
    title='Clusters (PCA Data)',
    width=500,
    height=500
)

# Display both side-by-side
original_cluster_plot + pca_cluster_plot

:Layout
   .NdOverlay.I  :NdOverlay   [cluster]
      :Scatter   [price_change_percentage_24h]   (price_change_percentage_7d,coin_id)
   .NdOverlay.II :NdOverlay   [cluster]
      :Scatter   [PC1]   (PC2,coin_id)

#### Answer the following question: 

  * **Question:** After visually analyzing the cluster analysis results, what is the impact of using fewer features to cluster the data using K-Means?

  * **Answer:** Using fewer features through PCA did not significantly impact the clustering results. The optimal number of clusters remained the same (k=3), and the overall cluster shapes and groupings were very similar between the original scaled data and the PCA-reduced data. This indicates that PCA successfully preserved the essential structure of the data while simplifying the feature space. It also made the data easier to visualize and may help improve performance in larger or noisier datasets.